# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [74]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import re
import nltk
nltk.download(['punkt', 'wordnet'])
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report
import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [44]:
# load data from database
engine = create_engine('sqlite:////home/workspace/disasterResponse.db')
df = pd.read_sql('disasterResponse', engine)

df.head()
X = df['message']
y = df.drop(['id', 'message', 'original', 'genre'], axis=1)

### 2. Write a tokenization function to process your text data

In [45]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

def tokenize(text):
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, 'urlplaceholder')
    
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
        
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [46]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [47]:
X_train, X_test, y_train, y_test = train_test_split(X,y)
#train classifier
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [52]:
#predict on the test data
y_pred = pipeline.predict(X_test)

#classification report
def each_col_report(y_test, y_pred):
    for i, col in enumerate(y_test.columns): 
        print('---------------------------------')
        print(col)
        print(classification_report(y_test.iloc[:,i], y_pred[:,i]))

each_col_report(y_test, y_pred)

---------------------------------
related
             precision    recall  f1-score   support

          0       0.64      0.35      0.45      1494
          1       0.82      0.94      0.88      5006
          2       0.78      0.13      0.22        54

avg / total       0.78      0.80      0.78      6554

---------------------------------
request
             precision    recall  f1-score   support

          0       0.88      0.98      0.93      5414
          1       0.81      0.37      0.51      1140

avg / total       0.87      0.88      0.86      6554

---------------------------------
offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6526
          1       0.00      0.00      0.00        28

avg / total       0.99      1.00      0.99      6554

---------------------------------
aid_related
             precision    recall  f1-score   support

          0       0.73      0.88      0.80      3792
          1       0.78   

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [56]:
parameters = {
    'tfidf__norm':['l2','l1'],
    'clf__estimator__min_samples_split':[2,3],
    }

cv = GridSearchCV(pipeline, parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [57]:
cv.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'tfidf__norm': ['l2', 'l1'], 'clf__estimator__min_samples_split': [2, 3]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [58]:
y_pred = cv.predict(X_test)

In [61]:
accuracy = (y_pred == y_test).sum()/y_test.shape[0]
print('accuracy :', accuracy)
each_col_report(y_test, y_pred)

accuracy : related                   0.797376
request                   0.872139
offer                     0.995728
aid_related               0.726732
medical_help              0.924321
medical_products          0.949191
search_and_rescue         0.972688
security                  0.982453
military                  0.966585
child_alone               1.000000
water                     0.947208
food                      0.915777
shelter                   0.922032
clothing                  0.988251
money                     0.977571
missing_people            0.988404
refugees                  0.967501
death                     0.958651
other_aid                 0.868172
infrastructure_related    0.931797
transport                 0.961092
buildings                 0.949954
electricity               0.977876
tools                     0.992676
hospitals                 0.990540
shops                     0.994355
aid_centers               0.987946
other_infrastructure      0.953464
weather_r

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [72]:
improved_pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

        ])),

        ('clf', MultiOutputClassifier(AdaBoostClassifier(n_estimators = 100)))
    ])


improved_pipeline.fit(X_train, y_train)
y_pred_improved = improved_pipeline.predict(X_test)
each_col_report(y_test, y_pred_improved)

---------------------------------
related
             precision    recall  f1-score   support

          0       0.43      0.38      0.40      1494
          1       0.82      0.85      0.83      5006
          2       0.50      0.22      0.31        54

avg / total       0.73      0.74      0.73      6554

---------------------------------
request
             precision    recall  f1-score   support

          0       0.91      0.96      0.94      5414
          1       0.77      0.55      0.64      1140

avg / total       0.89      0.89      0.89      6554

---------------------------------
offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6526
          1       0.00      0.00      0.00        28

avg / total       0.99      0.99      0.99      6554

---------------------------------
aid_related
             precision    recall  f1-score   support

          0       0.77      0.86      0.81      3792
          1       0.77   

### 9. Export your model as a pickle file

In [81]:
# save the model to disk
pickle.dump(improved_pipeline, open('model.sav', 'wb'))
 
# some time later...
 
# load the model from disk
loaded_model = pickle.load(open('model.sav', 'rb'))
result = loaded_model.score(X_test, y_test)
print(result)

0.242294781813


In [82]:
# save the model to disk
pickle.dump(pipeline, open('first_model.sav', 'wb'))
 
# some time later...
 
# load the model from disk
loaded_model = pickle.load(open('model.sav', 'rb'))
result = loaded_model.score(X_test, y_test)
print(result)

0.242294781813


### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.